# Scoring system with BETO

## Summary of the approach in this notebook

**Choice of BETO:** We choose BETO, a BERT model trained on Spanish corpora, because our text data is in Spanish and we need a model that understands the semantic meaning of sentences. BERT models are great at this because they're designed to understand the context of words in a sentence. By using BETO, we capture more complex language features compared to simpler techniques like TF-IDF.

**Generating Embeddings:** We use BETO to generate embeddings for both the senators' initiative profiles and the user's input. These embeddings are vectors in a high-dimensional space that represent the semantic meaning of the text. By representing the text as vectors, we can calculate the distance (or similarity) between different pieces of text.

**Calculating Similarity:** To match the user's input to the senators' profiles, we calculate the cosine similarity between the user's input vector and each senator's vector. This gives us a measure of how similar the user's input is to each senator's profile, which we use as our scoring mechanism.

**Ranking Senators:** Finally, we rank the senators based on their similarity scores and return the top N senators. This gives us a list of senators whose initiative profiles best match the user's interests.

The core idea of this approach is to leverage the power of language models like BETO to understand the semantic meaning of text and use this to match users with senators based on their interests.

## Import Libraries

In [2]:
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
import time
import pickle

## Initialize testing & cleaning DF

This dataframe is to test the functions in this notebook with the summarized data obtained from the BETO model. We also remove those senators that have no iniatives and we erase initials given by BETO ([SEP], [UNK], etc) to avoid problems when tokenizing & embedding the model.

In [5]:
#SENATORS_TO_PROCESS = 10

current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)
project_data_path = os.path.join(parent_directory, 'data')


senators_test_df_ini = pd.read_csv(os.path.join(project_data_path, 'senators_data_updated.csv')
                               ,converters={'BETO_summary': pd.eval}, index_col='Unnamed: 0')

In [6]:
valores_nulos_age = senators_test_df_ini['initiatives_summary_dummy'].isnull()
senators_test_df = senators_test_df_ini[valores_nulos_age == False]
print(senators_test_df.shape)
senators_test_df.head(2)

(124, 26)


,senator_id,Apellidos,Nombre,Fraccion,Legislatura,Estado,Sexo,tipoEleccion,Suplente,estadoOrigen,...,telefono,extension,estatus,direccion,id,senadores,attendance_score,initiative_list,initiatives_summary_dummy,BETO_summary
0,1160,Botello Montes,José Alfredo,PAN,64,Querétaro,Hombre,Senador Electo por el Principio de Mayoría Rel...,NaN,Querétaro,...,5345 3000,4183 y 4184,en_funciones,"Av Paseo de la Reforma No. 135, Hemiciclo Piso...",1,José Alfredo Botello Montes,0.958904,['Pretende garantizar a los miembros del Servi...,Pretende garantizar a los miembros del Servici...,[[CLS] Pretende garantizar a los miembros del ...
1,1162,Rojas Loreto,Estrella,PAN,64,Querétaro,Mujer,Senadora Electa por el Principio de Mayoría Re...,NaN,Querétaro,...,5345 3000,3323,en_funciones,"Av Paseo de la Reforma No. 135, Hemiciclo Piso...",2,Estrella Rojas Loreto,0.973913,['Propone que el Estado apoye la investigación...,Propone que el Estado apoye la investigación e...,[[CLS] Propone que el Estado apoye la investig...


In [7]:
senators_test_df['BETO_clean'] = senators_test_df['BETO_summary'].apply(lambda x: ' '.join(x))

/tmp/ipykernel_1995/1019380338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senators_test_df['BETO_clean'] = senators_test_df['BETO_summary'].apply(lambda x: ' '.join(x))


In [8]:
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('CLS', '')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('[', '')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace(']', '')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('SEP', '.')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('UNK', '.')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('  ', '')
senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace(' . ', ' ')

/tmp/ipykernel_1995/3518612301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('CLS', '')
/tmp/ipykernel_1995/3518612301.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  senators_test_df['BETO_clean'] = senators_test_df['BETO_clean'].str.replace('[', '')
/tmp/ipykernel_1995/3518612301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

## Download BETO model and tokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
model = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

### Test BETO

In [10]:
# Tokenize the text
inputs = tokenizer("Arriba la democracia!", return_tensors="pt")

# Generate the embeddings
outputs = model(**inputs)

# The embeddings are stored in the `last_hidden_state` attribute
embeddings = outputs.last_hidden_state
embeddings

tensor([[[ 0.0733, -0.2156,  0.1107,  ..., -0.1891,  0.1999,  0.1254],
         [-0.0935,  0.1006, -0.4794,  ..., -0.1307,  0.3479,  0.3917],
         [-0.2780,  1.1009, -0.6256,  ...,  0.0978,  0.3362,  0.4117],
         [-0.5546, -0.0858, -0.3468,  ...,  0.1149,  0.3315, -0.0632],
         [-0.4995, -0.1995, -0.0398,  ..., -0.3834,  0.0416, -0.3520],
         [-0.6204, -0.1591, -0.4836,  ..., -0.8248,  0.9292, -0.4672]]],
       grad_fn=<NativeLayerNormBackward0>)

## Generate embeddings with BETO

Embeddings are a way to represent text (or other types of data) as vectors of numbers. The key idea behind embeddings is to represent words or sentences in a high-dimensional space in such a way that their location in this space captures some of the semantic meaning of the text.

For example, in a well-constructed embedding space, words or sentences with similar meanings will be located near each other, and their relative locations can capture some of the relationships between them. For instance, the vectors for "king" and "queen" might be located at similar positions in the embedding space, and the direction from "king" to "queen" might be the same as the direction from "man" to "woman", capturing the relationship of gender between these words.

We use embeddings in NLP (Natural Language Processing) because they provide a way to turn text into a form that machine learning algorithms can understand. Most machine learning algorithms require numerical input, and embeddings provide a way to turn text into numbers while preserving some of the semantic meaning of the text.

In the context of this project, we use embeddings to represent both the user's input and the senator's initiative profiles. By representing these texts as vectors in a high-dimensional space, we can calculate the distance (or similarity) between the user's input and each senator's profile. This allows us to rank the senators based on how similar their profile is to the user's input.

In [11]:
def generate_embeddings(text, tokenizer, model, max_length=512):
    # Split the text into chunks to handle long summaries
    text_chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    
    # Initialize an empty tensor to store the embeddings
    embeddings = torch.zeros((len(text_chunks), model.config.hidden_size))
    
    # Generate embeddings for each chunk
    for i, chunk in enumerate(text_chunks):
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        embeddings[i] = outputs.last_hidden_state.mean(dim=1)
    
    # Average the embeddings of all the chunks
    embeddings = embeddings.mean(dim=0)
    
    return embeddings


### Test embeddings function

In [ ]:
#senators_test_df

In [ ]:
#senators_test_df['BETO_clean'].apply(lambda x: generate_embeddings(x, tokenizer, model))

### Why we don't use stop words or lemmatization?

In traditional NLP tasks, lemmatization and removing stop words are common steps to reduce the dimensionality of the data and focus on the most informative words. However, BERT-like models are based on transformers that use the context of words in a sentence to understand their meaning. They can even leverage the information contained in stop words. So, for these models, we usually keep the original form of the words and do not remove stop words.

## Score senator profiles based on user input

### Function to get scores

In [12]:
# Define a function to match senators based on the user's input
def match_senators(user_input, senators_embeddings, tokenizer, model):
    # Generate an embedding for the user's input
    user_embedding = generate_embeddings(user_input, tokenizer, model)

    # Initialize an empty array to store the similarity scores
    scores = np.zeros(len(senators_embeddings))

    # Calculate the cosine similarity between the user's input and each senator's profile
    for i, senator_embedding in enumerate(senators_embeddings):
        scores[i] = cosine_similarity(user_embedding.detach().numpy().reshape(1, -1), senator_embedding.detach().numpy().reshape(1, -1))

    return scores

### Test scoring function & loading pickle file

We ran the embedding function on a Virtual Machine, since the memory of a standard computer can't handle the amount of processing power needed for this function to work. We saved the output as a pickle file, which is loaded in the following cell, and we only need to run this embedding function every time we add more information to each senator.

In [14]:
with open(os.path.join(project_data_path,'Beto_embeddings.pkl'), 'rb') as handle:
    embeddings_final = pickle.load(handle)

In [15]:
# Get the user's input
user_input = "Quiero proteccion para los animales"

# Match the senators
similarity_scores = match_senators(user_input, embeddings_final, tokenizer, model)
similarity_scores

array([0.84296739, 0.84421575, 0.84330648, 0.85103428, 0.84016252,
       0.84657598, 0.8395682 , 0.84504509, 0.83749068, 0.83514267,
       0.84369707, 0.84133255, 0.84868652, 0.83283782, 0.84197783,
       0.84127116, 0.84597921, 0.84126824, 0.8481102 , 0.83960056,
       0.84715551, 0.84508675, 0.80708009, 0.84063071, 0.84621322,
       0.83618832, 0.84525275, 0.847781  , 0.84758502, 0.84529829,
       0.84857845, 0.84461367, 0.8436113 , 0.84727108, 0.84473228,
       0.81587398, 0.84524602, 0.83965784, 0.85054642, 0.85271806,
       0.84490967, 0.84552014, 0.85052019, 0.84612077, 0.85386539,
       0.843198  , 0.84653533, 0.84456164, 0.85078174, 0.83858562,
       0.85172045, 0.84876984, 0.8481409 , 0.84852898, 0.84284806,
       0.84859514, 0.84308547, 0.84894216, 0.83036655, 0.84549892,
       0.84928566, 0.85000885, 0.85235888, 0.85027051, 0.85106492,
       0.85036659, 0.85106784, 0.84645444, 0.83385104, 0.84571767,
       0.85055774, 0.8522979 , 0.85347223, 0.84851229, 0.84839

## Get top senators

### Function to get the senator with best matching score

In [16]:
def get_top_senators(scores, senators_df, N=5):
    # Get the indices of the senators sorted by their scores
    sorted_indices = np.argsort(scores)[::-1]
    
    # Get the top N senators
    top_senators = senators_df.iloc[sorted_indices[:N]]
    
    return top_senators

### Test top senators function

In [17]:
# Get the top 5 senators
top_senators = get_top_senators(similarity_scores, senators_test_df)

# Print the names and scores of the top senators
for i, row in top_senators.iterrows():
    print(f"Senator: {row['Nombre']} {row['Apellidos']}, Score: {similarity_scores[i]}")

Senator: Raúl de Jesús Elenes Angulo, Score: 0.8617692589759827
Senator: Raúl Paz Alonzo, Score: 0.8490737676620483
Senator: Alejandro Armenta Mier, Score: 0.8466907143592834
Senator: Oscar Eduardo Ramírez Aguilar, Score: 0.8471488952636719
Senator: Nancy Guadalupe Sánchez Arredondo, Score: 0.8495786786079407
